In [29]:
import gdxpds
import pandas as pd
import numpy as np
#import random

#Module that provides a portable way of using operating system dependent functionality
import os

from itertools import permutations, repeat

#Module to interact with the Notebook environment 
#https://ipywidgets.readthedocs.io/en/stable/
import ipywidgets as widgets

#Module for character encoding auto-detection 
#https://chardet.readthedocs.io/en/latest/index.html
import chardet 

#Plotting modules
import matplotlib
from matplotlib import pyplot as plt
#Allows plots to be shown on the Notebook
%matplotlib inline 

#Data visualization module based on matplotlib
#https://seaborn.pydata.org/
import seaborn as sns

#Cross-platform colored terminal text 
#https://pypi.org/project/colorama/
from colorama import Fore
from colorama import Style

from pandas.io.parsers import ParserError

In [79]:
dir_source = "./Data/Table_2"
dir_parsed = "./Results"

In [80]:
#Creating a list with the names of the data tables stored in the source directory
data_tables = os.listdir(dir_source)

#Creating a empty dictionary to store the data from the csv files
tables = {}

#Creating a list with the dictionary key`s names
key_names = ["table_2", "table_3"]


count = 0

#Looping through the csv files and uploading the data to the `Tables` dictionary
for fn in data_tables:
    name = key_names[count]
    path = dir_source + "/" + fn

    try:
        tables[name] = pd.read_csv(path, sep = "\,", engine='python') 
    except ParserError:
        tables[name] = pd.read_excel(path,engine = 'xlrd')
    except IndexError:
        pass
        
        
    count += 1
    
print(data_tables)    
print(tables)

['swisstore_table2_unige_may2019.csv', 'swisstore_table3_unige_may2019.csv']
{'table_2':        egid   age buildingtype typology canton hvacrtype  area_m2   r_chx  \
0       114  1970          SFH    Urban     ZH       Oil      150  676500   
1       199  1980          SFH    Urban     ZH   Electro      155  676500   
2       733  1980          MFH    Urban     ZH        HP      820  676500   
3       791  1920          MFH    Urban     ZH       Oil      223  676500   
4      1152  1945          SFH    Urban     ZH        HP      172  676500   
5      1164  1980          SFH    Urban     ZH   Electro      139  676500   
6   3022255  1990          SFH    Urban     ZH        HP      179  676500   
7   3022284  1990          SFH    Urban     ZH       Oil      230  676500   
8        50  1970          SFH    Urban     ZH       Oil      233  676500   
9       107  1980          SFH    Urban     ZH       Oil      175  676500   
10      112  1970          SFH    Urban     ZH       Oil      15

In [81]:
def archetype(df):
    """Returns a DataFrame with a column named "archetype", aggregating the data from the 
    "age", "buildingtype" and "typology" columns. The format is "year of construction-building type-typology"
    It also removes the source columns
    :param: df: <DataFrame> unedited Table 2 or 3
    return: df: <DataFrame> Edited Tables with the column "Archetype" """
    
    df["archetype"] = df["age"].map(str) + "-" + df["buildingtype"].map(str) + "-" \
                    + df["typology"].map(str)
    
    #df = df.drop(columns = ["age","buildingtype","typology"])
    
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]
    
    return df

#Creating a data frame for each table. For tables 2 and 3 the above function `archetype` 
#is applied
#table_1 = tables.get("1_demand_profiles")
table_2 = archetype(tables.get("table_2"))
print(table_2)
table_3 = archetype(tables.get("table_3"))
print(table_2)
#table_3 = tables.get("3_retrofits")
#print(table_3)
#table_4 = tables.get("4_new_technologies").dropna()
#print(table_4)
#table_5 = tables.get("5_energy_costs")
#print(table_5)

#Creating a list of buildings by ID
#id_list = list(pd.unique(table_1["egid"].tolist()))
#print(id_list)



#Creating a list of new technologies from Table 4
#tech_list = list(pd.unique(table_4["ID"].tolist()))
#tech_elec =[]
#tech_oil = []

#for tech in tech_list:
#    t = tech[0:5]
#    if t == "TH001":
#        tech_oil.append(tech)
#    else:
#        tech_elec.append(tech)
    
#print("tech_list")
#print(tech_list)
#print("tech_oil")
#print(tech_oil)
#print("tech_elec")
#print(tech_elec)

         archetype     egid   age buildingtype typology canton hvacrtype  \
0   1970-SFH-Urban      114  1970          SFH    Urban     ZH       Oil   
1   1980-SFH-Urban      199  1980          SFH    Urban     ZH   Electro   
2   1980-MFH-Urban      733  1980          MFH    Urban     ZH        HP   
3   1920-MFH-Urban      791  1920          MFH    Urban     ZH       Oil   
4   1945-SFH-Urban     1152  1945          SFH    Urban     ZH        HP   
5   1980-SFH-Urban     1164  1980          SFH    Urban     ZH   Electro   
6   1990-SFH-Urban  3022255  1990          SFH    Urban     ZH        HP   
7   1990-SFH-Urban  3022284  1990          SFH    Urban     ZH       Oil   
8   1970-SFH-Urban       50  1970          SFH    Urban     ZH       Oil   
9   1980-SFH-Urban      107  1980          SFH    Urban     ZH       Oil   
10  1970-SFH-Urban      112  1970          SFH    Urban     ZH       Oil   
11  1970-SFH-Urban      116  1970          SFH    Urban     ZH        HP   
12  1950-MFH

In [82]:
#Creating an empty dataframe to copy the desired data from table_2 and table_3
table_x = pd.DataFrame()

#Creating a list of all archetypes
arch_list = list(pd.unique(table_2["archetype"].tolist()))
print(arch_list)

for arch in arch_list:
    
    x = table_2.loc[table_2["archetype"] == arch]
    x.insert(10,"investment_cost__chf_per_m2",table_3.loc[table_3["archetype"] == arch, "investment_cost__chf_per_m2"].item())
    x.insert(11,"savings_relative",table_3.loc[table_3["archetype"] == arch, "savings_relative"].item())
    table_x = table_x.append(x)

table_2 = table_x
print(table_2) 

['1970-SFH-Urban', '1980-SFH-Urban', '1980-MFH-Urban', '1920-MFH-Urban', '1945-SFH-Urban', '1990-SFH-Urban', '1950-MFH-Urban', '1920-SFH-Urban']
         archetype     egid   age buildingtype typology canton hvacrtype  \
0   1970-SFH-Urban      114  1970          SFH    Urban     ZH       Oil   
8   1970-SFH-Urban       50  1970          SFH    Urban     ZH       Oil   
10  1970-SFH-Urban      112  1970          SFH    Urban     ZH       Oil   
11  1970-SFH-Urban      116  1970          SFH    Urban     ZH        HP   
13  1970-SFH-Urban      207  1970          SFH    Urban     ZH   Electro   
1   1980-SFH-Urban      199  1980          SFH    Urban     ZH   Electro   
5   1980-SFH-Urban     1164  1980          SFH    Urban     ZH   Electro   
9   1980-SFH-Urban      107  1980          SFH    Urban     ZH       Oil   
2   1980-MFH-Urban      733  1980          MFH    Urban     ZH        HP   
17  1980-MFH-Urban      838  1980          MFH    Urban     ZH       Oil   
18  1980-MFH-Urban 

In [84]:
import random
def rand(start, end, num): 
    '''A function to create a list of random numbers within a range.
    The quantity of numbers to be generated is also specified'''
    
    res = [] 
  
    for j in range(num): 
        res.append(random.randint(start, end)) 
  
    return res 


#Adding the new columns to table_2
table_2["arch_entry_id"] = list(range(1000001,1000021))
table_2["archetype_climate_id"] = list(range(1000001,1000021))
table_2["arch_id"] = list(range(1,21))
table_2["clim_id"] = list(range(2101,2121))
table_2["era_median"] = rand(50,1000,20)
table_2["era_avg"] = rand(50,1000,20)
table_2["tot_buildings"] = rand(1,25,20)
table_2["enable_borehole"] = rand(0,1,20)
table_2["enable_gas"] = rand(0,1,20)
table_2["enable_district_heat"] = rand(0,1,20)
table_2["ht_temp_level"] = rand(0,1,20)
table_2["dhw_decentral"] = rand(0,1,20)
table_2["p_therm_heating"] = rand(1,120,20)
table_2["p_therm_dhw"] = rand(0,4,20)
table_2["p_therm_tot"] = rand(9,120,20)
table_2["p_power_dec"] = rand(0,10,20)
table_2["dhw_power_dec"] = rand(0,10,20)
table_2["dhw_storage_decentral"] = rand(0,35,20)
table_2["dhw_daily_energy"] = rand(30.0,60.0,20)

#Renaming the columns that are common to dummy tables and the initialization process table
table_2.rename(columns={'age':'constr_period','buildingtype':'build_type',\
                        'typology':'area_typology','hvacrtype':'hvac_system',\
                         'usefulenergy_annual__kwh':'energy_specific'}, inplace=True)

#Dropping unecessary columns

#table_2 = table_2.drop(columns = ['archetype','egid','area_m2','r_chx','r_chy',\
#                                  'investment_cost__chf_per_m2','savings_relative',\
#                                  'finalenergy_annual__kwh','primaryenergy_annual__kwh'])


#Reordering the order of the table's columns 

table_2 = table_2[['arch_entry_id','archetype_climate_id','arch_id','clim_id','constr_period',\
                   'build_type','area_typology','canton','hvac_system','era_median',\
                   'era_avg', 'tot_buildings','energy_specific','enable_borehole',\
                   'enable_gas','enable_district_heat','ht_temp_level','dhw_decentral',\
                   'dhw_daily_energy','dhw_storage_decentral','dhw_power_dec',\
                   'p_therm_heating','p_therm_dhw','p_therm_tot']]


In [85]:
table_2.head(20)

,arch_entry_id,archetype_climate_id,arch_id,clim_id,constr_period,build_type,area_typology,canton,hvac_system,era_median,...,enable_gas,enable_district_heat,ht_temp_level,dhw_decentral,dhw_daily_energy,dhw_storage_decentral,dhw_power_dec,p_therm_heating,p_therm_dhw,p_therm_tot
0,1000001,1000001,1,2101,1970,SFH,Urban,ZH,Oil,660,...,1,0,1,0,49,35,2,35,0,10
8,1000002,1000002,2,2102,1970,SFH,Urban,ZH,Oil,190,...,0,0,1,0,58,18,9,111,2,117
10,1000003,1000003,3,2103,1970,SFH,Urban,ZH,Oil,990,...,1,0,1,1,52,5,9,111,4,23
11,1000004,1000004,4,2104,1970,SFH,Urban,ZH,HP,973,...,1,0,0,1,45,25,1,119,4,18
13,1000005,1000005,5,2105,1970,SFH,Urban,ZH,Electro,487,...,0,1,1,0,42,33,5,116,3,61
1,1000006,1000006,6,2106,1980,SFH,Urban,ZH,Electro,351,...,1,0,1,1,56,11,8,35,1,56
5,1000007,1000007,7,2107,1980,SFH,Urban,ZH,Electro,537,...,1,0,0,0,43,13,2,9,1,65
9,1000008,1000008,8,2108,1980,SFH,Urban,ZH,Oil,903,...,1,0,1,1,50,15,7,82,3,89
2,1000009,1000009,9,2109,1980,MFH,Urban,ZH,HP,638,...,1,1,1,1,46,16,4,6,4,51
17,1000010,1000010,10,2110,1980,MFH,Urban,ZH,Oil,395,...,1,1,0,0,58,23,4,117,1,86


In [86]:
#Exporting table_2 to a csv file

dir_csv = dir_parsed + "/" + 'table_2' + ".csv"

table_2.to_csv(dir_csv, sep = ',', index = False)



In [87]:
def download(d_name,dict_key,ID_key,dict_dict):
    
    for name, value in dict_dict.items():
        if name == d_name:
            dir_csv = dir_parsed + "/" + dict_key + "_" + str(ID_key) + ".csv"
            print(dir_csv)
            value[dict_key][ID_key].to_csv(dir_csv, sep = ',', index = False) 